# Data Visualization with Plotly

The following notebook displays how the plotly suite may be used to visualize data using Python.

## Setting up

Before getting started make sure your python environment is set up correctly, we recommend using `virtualenv` (which also requires `pipx`) for maximum isolation of dependencies.

Restore the dependencies to run this project by running `pip install -r requirements.txt`.

## First hello world with Ploty

The code belows renders a very simple bar chart by using Plotly. Think of it as the basic "hello, world" that confirms your environment is indeed working as expected.

In [1]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

## Getting simple data from an API and visualizing it.

For this example I`ll use [FIPE API](https://deividfortuna.github.io/fipe/) as the source of our data.

The following snippet creates a class to describe a vehicle from this API, calls the API, parses the results into the objects of the Vehicle class and then plots information about the most expensive ones.

### Representing Vehicles using Object orientation

To follow best practices and make things more readable long term we'll be creating Classes to represent the objects that are returned from the API as well as an object to represent the API itself (and thus allow us to have an easier time calling it).

Since we are on an iterative notebook we'll be using the `assert` keyword to test things instead of a proper unit test. In production code we would have an accompanying unit test suite to cover the data contract and any API logic that'd be needed.

Also we'll be using the "type hinting" system from Python's latest versions to further document the objects we'll be creating and working with once we call the api.

A helper method (`from_json`) is created to make it simpler to instantiate objects of those classes from a json input.

In [23]:
import json #since we are working with json data
from typing import Self #for type hinting

class Make:
    """
    Represents a Make within the FIPE Api.

    Attributes:
      codigo: str - The identifier of the make for API calls.
      nome: str - The name of the make.
    """

    def __init__(self: Self, codigo: str, nome: str):
        """
        Initializes a Make object.
        
        Args:
          codigo: str - The identifier of the make for API calls.
          nome: str - The name of the make.
        """
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data) -> Self:
        """
        Creates a Make instance from json data.
        """
        return cls(**json_data)

class Model:
    """
    Represents a Model within the FIPE Api. Usually a result of a subsequent call after getting a Make.

    Attributes:
      codigo: int - The identifier of the model for API calls.
      nome: str - The name of the model.
    """

    def __init__(self: Self, codigo: int, nome: str):
        """
        Initializes a Model object.
        
        Args:
          codigo: int - The identifier of the model for API calls.
          nome: str - The name of the model.
        """
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data):
        """
        Creates a Model instance from json data.
        """
        return cls(**json_data)

class ModelYear:
    """
    Represent`s a ModelYear within the FIPE Api. Usually a subsequent call after getting a Model.

    Attributes:
      codigo: str - The identifier of the model year for API calls.
      nome: str - The name of the model year.
    """

    def __init__(self: Self, codigo: str, nome: str):
        """
        Initializes a ModelYear object.

        Args:
          codigo: str - The identifier of the model year for API calls.
          nome: str - The name of the model year.
        """
        self.codigo = codigo
        self.nome = nome

    @classmethod
    def from_json(cls, json_data) -> Self:
        return cls(**json_data)

class ModelYearPrice:
    """
    Represents a ModelYearPrice within the FIPE Api. Usually a subsequent call after getting a ModelYear.

    Attributes:
      TipoVeiculo: int - The type of vehicle (car, truck, motorcycle)
      Valor: str - The price of the vehicle in R$.
      Marca: str - The make of the vehicle.
      Modelo: str - The model of the vehicle.
      AnoModelo: int - The year of the model.
      Combustivel: str - The fuel type of the vehicle.
      CodigoFipe: str - The identifier of the price for API calls.
      MesReferencia: str - The month of the price.
      SiglaCombustivel: str - The fuel type abbreviation.
    """

    def __init__(self: Self, 
                 TipoVeiculo: int, 
                 Valor: str, 
                 Marca: str, 
                 Modelo: str, 
                 AnoModelo: int, 
                 Combustivel: str, 
                 CodigoFipe: str, 
                 MesReferencia: str, 
                 SiglaCombustivel: str):
        """
        Initializes a ModelYearPrice object.

        Args:
          TipoVeiculo: int - The type of vehicle (car, truck, motorcycle)
          Valor: str - The price of the vehicle in R$.
          Marca: str - The make of the vehicle.
          Modelo: str - The model of the vehicle.
          AnoModelo: int - The year of the model.
          Combustivel: str - The fuel type of the vehicle.
          CodigoFipe: str - The identifier of the price for API calls.
          MesReferencia: str - The month of the price.
          SiglaCombustivel: str - The fuel type abbreviation
        """
        self.TipoVeiculo = TipoVeiculo
        self.Valor = Valor
        self.Marca = Marca
        self.Modelo = Modelo
        self.AnoModelo = AnoModelo
        self.Combustivel = Combustivel
        self.CodigoFipe = CodigoFipe
        self.MesReferencia = MesReferencia
        self.SiglaCombustivel = SiglaCombustivel

    @classmethod
    def from_json(cls, json_data) -> Self:
        return cls(**json_data)

# testing if the make class is accurate
makeJson = "{\"codigo\":\"1\",\"nome\":\"Acura\"}"
makeData = json.loads(makeJson)
makeTest = Make.from_json(makeData)
assert makeTest.codigo == "1"
assert makeTest.nome == "Acura"

# testing if the model class is accurate
modelJson = "{\"codigo\":9985,\"nome\":\"Corolla Altis Prem. 1.8 Aut. (HÃ­brido)\"}" 
modelData = json.loads(modelJson)
modelTest = Model.from_json(modelData)
assert modelTest.codigo == 9985
assert modelTest.nome == "Corolla Altis Prem. 1.8 Aut. (HÃ­brido)"

# testing it the modelyear class is accurate
modelYearJson = "{\"codigo\":\"32000-1\",\"nome\":\"32000 Gasolina\"}"
modelYearData = json.loads(modelYearJson)
modelYearTest = ModelYear.from_json(modelYearData)
assert modelYearTest.codigo == "32000-1"
assert modelYearTest.nome == "32000 Gasolina"

# testing if the ModelYearPrice class is accurate
modelYearPriceJson = "{\"TipoVeiculo\":1,\"Valor\":\"R$ 140.970,00\",\"Marca\":\"Toyota\",\"Modelo\":\"Corolla Altis Prem. 1.8 Aut. (HÃ­brido)\",\"AnoModelo\":2022,\"Combustivel\":\"Gasolina\",\"CodigoFipe\":\"002183-0\",\"MesReferencia\":\"novembro de 2024\",\"SiglaCombustivel\":\"G\"}"
modelYearPriceData = json.loads(modelYearPriceJson)
modelYearPriceTest = ModelYearPrice.from_json(modelYearPriceData)
assert modelYearPriceTest.TipoVeiculo == 1
assert modelYearPriceTest.Valor == "R$ 140.970,00"
# not testing the rest of the attributes since they are the same as the ones in the json

### Building a Fluent class to Query the API

To make it easier to interact with the remote API we'll build a "Builder" class that handles the additional logic of building out the API endpoints following its logic:

1. the base URL must be called in order to get a make's id
2. the models endpoint must be called with the make's id in its path to get the available models
3. the years endpoint must be called with the make's id and the model's id in order to return the yearly entries for that model
4. finally the year's code is used to retrieve the actual price of the vehicle


In [22]:
# BrasilAPI endpoint with all car brands
makeEndpoint = "https://deividfortuna.github.io/fipe/"
modelsEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos"
yearsEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos/<model>/anos"
priceEndpoint = "https://parallelum.com.br/fipe/api/v1/carros/marcas/<make>/modelos/<model>/anos/<year>"

# TODO; WIP